<a href="https://colab.research.google.com/github/Hyenni/kaggle/blob/cancer/cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## kaggle competition data download method

In [0]:
!pip install kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       2600           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      10047            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       4105           False  
imagenet-object-localization-challenge         2029-12-31 07:00:00  Research         Knowledge         33           False  
competitive-data-science-predict-future-sales  2019-12-31 23:59:00  Playground           Kudos       2305           False  
two-sigma-financial-news                       2019-07-15 23:59:00  Featured          $100,000       2897           False  
LANL-Ear

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c histopathologic-cancer-detection

  0% 0.00/1.33M [00:00<?, ?B/s]
100% 1.33M/1.33M [00:00<00:00, 44.5MB/s]
 98% 5.00M/5.10M [00:00<00:00, 20.8MB/s]
100% 5.10M/5.10M [00:00<00:00, 20.3MB/s]
100% 1.30G/1.30G [00:20<00:00, 41.6MB/s]
100% 1.30G/1.30G [00:20<00:00, 69.1MB/s]
100% 4.98G/4.98G [01:24<00:00, 75.2MB/s]
100% 4.98G/4.98G [01:25<00:00, 62.9MB/s]


In [0]:
!ls -al

total 6596528
drwxr-xr-x 1 root root       4096 Feb 11 04:49 .
drwxr-xr-x 1 root root       4096 Feb 11 04:46 ..
drwxr-xr-x 1 root root       4096 Feb  6 17:30 .config
-rw-r--r-- 1 root root         62 Feb 11 04:48 kaggle.json
drwxr-xr-x 1 root root       4096 Feb  6 17:31 sample_data
-rw-r--r-- 1 root root    1394488 Feb 11 04:49 sample_submission.csv.zip
-rw-r--r-- 1 root root 1401100547 Feb 11 04:49 test.zip
-rw-r--r-- 1 root root    5352900 Feb 11 04:49 train_labels.csv.zip
-rw-r--r-- 1 root root 5346961539 Feb 11 04:51 train.zip


In [0]:
!unzip sample_submission.csv.zip

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   


In [0]:
!unzip train_labels.csv.zip

Archive:  train_labels.csv.zip
  inflating: train_labels.csv        


In [0]:
!ls -al

total 6608184
drwxr-xr-x 1 root root       4096 Feb 11 04:51 .
drwxr-xr-x 1 root root       4096 Feb 11 04:46 ..
drwxr-xr-x 1 root root       4096 Feb  6 17:30 .config
-rw-r--r-- 1 root root         62 Feb 11 04:48 kaggle.json
drwxr-xr-x 1 root root       4096 Feb  6 17:31 sample_data
---------- 1 root root    2470703 Nov 15 23:48 sample_submission.csv
-rw-r--r-- 1 root root    1394488 Feb 11 04:49 sample_submission.csv.zip
-rw-r--r-- 1 root root 1401100547 Feb 11 04:49 test.zip
---------- 1 root root    9461084 Nov 15 23:48 train_labels.csv
-rw-r--r-- 1 root root    5352900 Feb 11 04:49 train_labels.csv.zip
-rw-r--r-- 1 root root 5346961539 Feb 11 04:51 train.zip


In [0]:
!unzip train.zip

In [0]:
!unzip test.zip



---

---




In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, Input, Conv2D, MaxPooling2D, BatchNormalization, Concatenate, ReLU, LeakyReLU
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras import metrics
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import keras
import tensorflow as tf

from tensorflow import set_random_seed
set_random_seed(777)

import pandas as pd
import numpy as np

import os
import cv2

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

SHAPE = (96,96,3)
BATCH_SIZE = 64

![1](https://user-images.githubusercontent.com/33708512/52691262-ae2efb00-2fa3-11e9-8056-1ef43dac2882.PNG)

## * File load

### Label as per train_labels.csv file

0 = no tumor tissue<br>
1 =   has tumor tissue. <br>

Label이 0인 이미지는 암세포가 없으며, 1인 이미지는 암세포가 있습니다.

In [0]:
df_data = pd.read_csv('../input/train_labels.csv')
print(df_data.head())

![1](https://user-images.githubusercontent.com/33708512/52547214-65dbd580-2e09-11e9-8f35-448a4da3e57b.PNG)

### Create a Dataframe containing all images
아래 코드는 train_labels.csv 파일을 읽어들이고, 이전에 에러를 일으켰던 이미지와 검은 이미지를 삭제합니다.

In [0]:

# removing this image because it caused a training error previously
#아래 이미지는 이전에 에러를 일으켰기 때문에 삭제합니다.
df_data = df_data[df_data['id'] != 'dd6dfed324f9fcb6f93f46f32fc800f2ec196be2']

# removing this image because it's black
#아래 이미지는 그냥 검은 이미지라서 삭제합니다.
df_data = df_data[df_data['id'] != '9369c7278ec8bcc6c880d99194de09fc2bd4efbe']


print(df_data.shape)

![3](https://user-images.githubusercontent.com/33708512/52691379-05cd6680-2fa4-11e9-9059-4541f0e6a907.PNG)

In [0]:
print('Number of image : ', len(df_data))
print('Ratio labels : ', sum(df_data['label'].values)/len(df_data))
img = plt.imread("../input/train/"+df_data.iloc[0]['id']+'.tif')
print('Images shape', img.shape)

![4](https://user-images.githubusercontent.com/33708512/52691442-2ac1d980-2fa4-11e9-96b1-8c531c512d4f.PNG)

## * View the data distribution

In [0]:
import seaborn as sns
sns.countplot('label', data=df_data)

![5](https://user-images.githubusercontent.com/33708512/52691443-2b5a7000-2fa4-11e9-826a-c098082d2986.PNG)


In [0]:
df_data['label'].value_counts()

![6](https://user-images.githubusercontent.com/33708512/52691444-2b5a7000-2fa4-11e9-8dd9-d5b518b142c1.PNG)

In [0]:
df_data.head(10)

![7](https://user-images.githubusercontent.com/33708512/52691445-2b5a7000-2fa4-11e9-9b6b-b2cc0f68400d.PNG)

## * Display a random sample of train images by class

In [0]:
# source: https://www.kaggle.com/gpreda/honey-bee-subspecies-classification

def draw_category_images(col_name,figure_cols, df, IMAGE_PATH):

    categories = (df.groupby([col_name])[col_name].nunique()).index
    f, ax = plt.subplots(nrows=len(categories),ncols=figure_cols, 
                         figsize=(4*figure_cols,4*len(categories))) # adjust size here
    # draw a number of images for each location
    for i, cat in enumerate(categories):
        sample = df[df[col_name]==cat].sample(figure_cols) # figure_cols is also the sample size
        for j in range(0,figure_cols):
            file=IMAGE_PATH + sample.iloc[j]['id'] + '.tif'
            im=cv2.imread(file)
            ax[i, j].imshow(im, resample=True, cmap='gray')
            ax[i, j].set_title(cat, fontsize=16)  
    plt.tight_layout()
    plt.show()

In [0]:
IMAGE_PATH = '../input/train/' 

draw_category_images('label',4, df_data, IMAGE_PATH)

![8](https://user-images.githubusercontent.com/33708512/52691446-2b5a7000-2fa4-11e9-988d-b4c347e9e317.PNG)

### End of Data Preparation <br> ==================================================================================== <br> Start of Model Building

## * Set Up the Generators
### 모델 설계에 앞서, Data Generator를 만들어야 합니다.


In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.1
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_path = '../input/train'
valid_path = '../input/train'

train_generator = train_datagen.flow_from_dataframe(
                dataframe=df_data,
                directory=train_path,
                x_col = 'id',
                y_col = 'label',
                has_ext=False,
                subset='training',
                target_size=(96, 96),
                batch_size=64,
                class_mode='binary'
                )

validation_generator = train_datagen.flow_from_dataframe(
                dataframe=df_data,
                directory=valid_path,
                x_col = 'id',
                y_col = 'label',
                has_ext=False,
                subset='validation',
                target_size=(96, 96),
                batch_size=64,
                shuffle=False,
                class_mode='binary'
                )

![9](https://user-images.githubusercontent.com/33708512/52691447-2bf30680-2fa4-11e9-9f50-fcead9dc9ae6.PNG)

## * Create the Model Architecture

In [0]:
def create_model(input_shape):
    
    dropRate = 0.25
    
    init = Input(input_shape)
    x = BatchNormalization(axis=-1)(init)
    x = Conv2D(8, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(8, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(16, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    c1 = Conv2D(16, (3, 3), padding='same')(x)
    c1 = ReLU()(c1)
    c2 = Conv2D(16, (5, 5), padding='same')(x)
    c2 = ReLU()(c2)
    c3 = Conv2D(16, (7, 7), padding='same')(x)
    c3 = ReLU()(c3)
    c4 = Conv2D(16, (1, 1), padding='same')(x)
    c4 = ReLU()(c4)
    x = Concatenate()([c1, c2, c3, c4])
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    x = Conv2D(32, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    x = Conv2D(64, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(28)(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(0.1)(x)
    x = Dense(1)(x)
    x = Activation('sigmoid')(x)
    
    model = Model(init, x)
    
    return model

## * Train the Model

In [0]:
model = create_model(SHAPE)
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(1e-03),
    metrics=['accuracy'])

model.summary()

![10-1](https://user-images.githubusercontent.com/33708512/52691449-2bf30680-2fa4-11e9-97d7-5afd4db9f7b3.PNG)
![10-2](https://user-images.githubusercontent.com/33708512/52691451-2c8b9d00-2fa4-11e9-80e1-e328c41a7675.PNG)
![10-3](https://user-images.githubusercontent.com/33708512/52691452-2c8b9d00-2fa4-11e9-9e36-5f22b8cee5de.PNG)
![10-4](https://user-images.githubusercontent.com/33708512/52691453-2c8b9d00-2fa4-11e9-9ee6-313aeaa46bb2.PNG)
![10-5](https://user-images.githubusercontent.com/33708512/52691454-2d243380-2fa4-11e9-9c6a-288f4ebefd48.PNG)
![10-6](https://user-images.githubusercontent.com/33708512/52691456-2d243380-2fa4-11e9-91b1-a5d6b0d1f449.PNG)
![10-7](https://user-images.githubusercontent.com/33708512/52691458-2d243380-2fa4-11e9-959f-d011e816b819.PNG)

In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

history = model.fit_generator(
                train_generator,
                steps_per_epoch=STEP_SIZE_TRAIN,
                validation_data=validation_generator,
                validation_steps=STEP_SIZE_VALID,epochs=20)

![11](https://user-images.githubusercontent.com/33708512/52691460-2dbcca00-2fa4-11e9-8080-a90c2aa4185c.PNG)

In [0]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 2, figsize=(15,5))
ax[0].set_title('loss')
ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
ax[1].set_title('acc')
ax[1].plot(history.epoch, history.history["acc"], label="Training acc")
ax[1].plot(history.epoch, history.history["val_acc"], label="Validation acc")
ax[0].legend()
ax[1].legend()

![12](https://user-images.githubusercontent.com/33708512/52691462-2dbcca00-2fa4-11e9-8c35-636511dfd9da.PNG)

## * Evaluate the model using the test set

In [0]:
# Kaggle testing
from glob import glob
from skimage.io import imread 

testing_files = glob(os.path.join('../input/test/','*.tif'))
submission = pd.DataFrame()
for index in range(0, len(testing_files), BATCH_SIZE):
    data_frame = pd.DataFrame({'path': testing_files[index:index+BATCH_SIZE]})
    data_frame['id'] = data_frame.path.map(lambda x: x.split('/')[3].split(".")[0])
    data_frame['image'] = data_frame['path'].map(imread)
    images = np.stack(data_frame.image, axis=0)
    predicted_labels = [model.predict(np.expand_dims(image/255.0, axis=0))[0][0] for image in images]
    predictions = np.array(predicted_labels)
    data_frame['label'] = predictions
    submission = pd.concat([submission, data_frame[["id", "label"]]])
submission.to_csv('./submission_epoch20.csv', index=False, header=True)

In [0]:
output = pd.read_csv('./submission_epoch20.csv')
output.head()

![13](https://user-images.githubusercontent.com/33708512/52691463-2dbcca00-2fa4-11e9-9158-1f14a1c3cf6c.PNG)

### Reference Kernels


*   https://www.kaggle.com/cheongwoongk/cnn-cancer-detection

